# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Elemento de lista



In [1]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from collections import Counter
import gensim.downloader as api

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import os
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sortega/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sortega/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sortega/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [2]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
current_dir = os.getcwd()

# Define the relative paths to the files
amazon_file_path = os.path.join(current_dir, 'amazon5.txt')
imdb_file_path = os.path.join(current_dir, 'imdb5.txt')
yelp_file_path = os.path.join(current_dir, 'yelp5.txt')

# Reading the files
amazon_df = pd.read_csv(amazon_file_path, sep='\t', header=None, names=['review', 'label'])
imdb_df = pd.read_csv(imdb_file_path, sep=r'\s{3}', header=None, names=['review', 'label'], engine='python')
yelp_df = pd.read_csv(yelp_file_path, sep='\t', header=None, names=['review', 'label'])



# Concatenate the three DataFrames into one
df = pd.concat([amazon_df, imdb_df, yelp_df], ignore_index=True)


# *********** Aquí termina la sección de agregar código *************


In [3]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [4]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [5]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
mystopwords = set(stopwords.words('english')).difference(negwords)
lemmatizer = WordNetLemmatizer()

def normalize_repeated_characters(text):
    return re.sub(r'(.)\1{2,}', r'\1', text)

def clean_and_lemmatize(text):
    # Normalize repeated characters
    normalized_text = normalize_repeated_characters(text)
    # Tokenize the text
    tokens = nltk.word_tokenize(normalized_text)
    # Remove stopwords, lemmatize, and remove short tokens
    lemmatized = [
        lemmatizer.lemmatize(word.lower())
        for word in tokens
        if word.lower() not in mystopwords and word.isalpha() and len(word) > 1
    ]
    return ' '.join(lemmatized)


X = df['review']
Y = df['label'].astype(int)

Xclean = X.apply(clean_and_lemmatize)

# *********** Aquí termina la sección de agregar código *************

In [6]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[40:45]:
  print(x)


great camera thats pic nice clear great picture quality
not impressed product
nice headset priced right
hear garbage audio
excellent bluetooth headset


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [7]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450
X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [8]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


midiccionario = Counter()

for text in x_train:
    tokens = [token for token in text.split() if token.strip()]
    midiccionario.update(tokens)


print('Longitud del diccionario:', len(midiccionario))
print('\n(word,frequency):')
print(midiccionario.most_common(10))


# *********** Aquí termina la sección de agregar código *************

Longitud del diccionario: 3578

(word,frequency):
[('not', 225), ('good', 160), ('movie', 140), ('great', 137), ('phone', 131), ('film', 125), ('time', 100), ('one', 99), ('like', 88), ('work', 86)]


In [9]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


# Imprimir la longitud del vocabulario generado
print(len(midiccionario))



# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:
3578


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

*Para evitar Data leakegae. De esta forma aseguramos que el modelo generalize bien y no se aprenda la información*

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [10]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


def filtrar_comentarios(conjunto, vocabulario):
    comentarios_filtrados = []
    for comentario in conjunto:
        tokens = [token for token in comentario.split() if token in vocabulario]
        comentarios_filtrados.append(' '.join(tokens))
    return comentarios_filtrados

numero_minimo_ocurrencias = 2

# Filtrar las palabras en el vocabulario
vocabulario_filtrado = {word for word, count in midiccionario.items() if count >= numero_minimo_ocurrencias}

print('Tamaño del vocabulario filtrado:', len(vocabulario_filtrado))

# Aplicar la función a los conjuntos de entrenamiento, validación y prueba
train_x = filtrar_comentarios(x_train, vocabulario_filtrado)
val_x = filtrar_comentarios(x_val, vocabulario_filtrado)
test_x = filtrar_comentarios(x_test, vocabulario_filtrado)


# *********** Aquí termina la sección de agregar código *************


Tamaño del vocabulario filtrado: 1506


In [11]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

fare much better people like morgan ed wasted
tonight filet sucked
paid bill not tip felt server terrible job
call ca properly cook steak understand
however keypad tinny sometimes wrong button


#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [12]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


fasttext_model = api.load('fasttext-wiki-news-subwords-300')




# *********** Aquí termina la sección de agregar código *************

In [13]:

print('Tamaño del vocabulario filtrado:', len(vocabulario_filtrado))
embedding_dict = {}

# Para cada palabra en el vocabulario filtrado, obtener su vector embebido
for word in vocabulario_filtrado:
    if word in fasttext_model:
        embedding_dict[word] = fasttext_model[word]
    else:
        print(f'No se encontró el vector embebido para la palabra "{word}"')

print(f'Tamaño del diccionario de vectores embebidos: {len(embedding_dict)}')

Tamaño del vocabulario filtrado: 1506
No se encontró el vector embebido para la palabra "razr"
No se encontró el vector embebido para la palabra "huston"
No se encontró el vector embebido para la palabra "jabra"
No se encontró el vector embebido para la palabra "errol"
No se encontró el vector embebido para la palabra "plantronics"
No se encontró el vector embebido para la palabra "schrader"
No se encontró el vector embebido para la palabra "miyazaki"
No se encontró el vector embebido para la palabra "vinegrette"
No se encontró el vector embebido para la palabra "fulci"
No se encontró el vector embebido para la palabra "mishima"
No se encontró el vector embebido para la palabra "stanwyck"
Tamaño del diccionario de vectores embebidos: 1495


In [14]:


# Guardar el diccionario en un archivo
with open('embedding_dict.pkl', 'wb') as file:
    pickle.dump(embedding_dict, file)

print('Diccionario de vectores embebidos guardado en embedding_dict.pkl')

Diccionario de vectores embebidos guardado en embedding_dict.pkl


In [15]:
del fasttext_model

#**Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [16]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

with open('embedding_dict.pkl', 'wb') as file:
    pickle.dump(embedding_dict, file)

print('Diccionario de vectores embebidos guardado en embedding_dict.pkl')

# Generate embedding vectors function with padding
def generar_vectores_embebidos(conjunto, embedding_dict, embedding_dim=300, max_length=None):
    vectores_embebidos = []
    for comentario in conjunto:
        tokens = comentario.split()
        vector_comentario = []
        for token in tokens:
            if token in embedding_dict:
                vector_comentario.append(embedding_dict[token])
            else:
                vector_comentario.append(np.zeros(embedding_dim))
        
        # Pad sequences to the maximum length
        if max_length:
            while len(vector_comentario) < max_length:
                vector_comentario.append(np.zeros(embedding_dim))
            vector_comentario = vector_comentario[:max_length]
        
        vectores_embebidos.append(np.array(vector_comentario))
    return np.array(vectores_embebidos)

# Determine the maximum length of the sequences
max_length = max(len(comentario.split()) for comentario in train_x + val_x + test_x)

# Generate embedding vectors for the datasets with padding
trainEmb = generar_vectores_embebidos(train_x, embedding_dict, max_length=max_length)
valEmb = generar_vectores_embebidos(val_x, embedding_dict, max_length=max_length)
testEmb = generar_vectores_embebidos(test_x, embedding_dict, max_length=max_length)

# Save the embedding vectors to files
with open('trainEmb.pkl', 'wb') as file:
    pickle.dump(trainEmb, file)
with open('valEmb.pkl', 'wb') as file:
    pickle.dump(valEmb, file)
with open('testEmb.pkl', 'wb') as file:
    pickle.dump(testEmb, file)

print('Vectores embebidos guardados en archivos pkl.')


# *********** Aquí termina la sección de agregar código *************

Diccionario de vectores embebidos guardado en embedding_dict.pkl
Vectores embebidos guardados en archivos pkl.


In [17]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (2100, 22, 300)
Val-Emb: (450, 22, 300)
Test-Emb: (450, 22, 300)


#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [25]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


from sklearn.linear_model import LogisticRegression

# Aplanar las matrices de vectores embebidos
X_train = np.array([x.flatten() for x in trainEmb])
X_val = np.array([x.flatten() for x in valEmb])
X_test = np.array([x.flatten() for x in testEmb])

# Entrenar el modelo de regresión logística
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Predecir y evaluar
y_pred_train = logreg.predict(X_train)
y_pred_val = logreg.predict(X_val)
y_pred_test = logreg.predict(X_test)

print("Regresión Logística - Desempeño en el conjunto de entrenamiento")
print(classification_report(y_train, y_pred_train))

print("Regresión Logística - Desempeño en el conjunto de validación")
print(classification_report(y_val, y_pred_val))

print("Regresión Logística - Desempeño en el conjunto de prueba")
print(classification_report(y_test, y_pred_test))



# *********** Aquí termina la sección de agregar código *************


Regresión Logística - Desempeño en el conjunto de entrenamiento
              precision    recall  f1-score   support

           0       0.91      0.94      0.93      1068
           1       0.94      0.91      0.92      1032

    accuracy                           0.92      2100
   macro avg       0.92      0.92      0.92      2100
weighted avg       0.92      0.92      0.92      2100

Regresión Logística - Desempeño en el conjunto de validación
              precision    recall  f1-score   support

           0       0.78      0.80      0.79       216
           1       0.81      0.79      0.80       234

    accuracy                           0.80       450
   macro avg       0.80      0.80      0.80       450
weighted avg       0.80      0.80      0.80       450

Regresión Logística - Desempeño en el conjunto de prueba
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       216
           1       0.83      0.81      0.82       234



In [19]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

# Predecir y evaluar
y_pred_train_rf = rf.predict(X_train)
y_pred_val_rf = rf.predict(X_val)
y_pred_test_rf = rf.predict(X_test)

print("Bosque Aleatorio - Desempeño en el conjunto de entrenamiento")
print(classification_report(y_train, y_pred_train_rf))

print("Bosque Aleatorio - Desempeño en el conjunto de validación")
print(classification_report(y_val, y_pred_val_rf))

print("Bosque Aleatorio - Desempeño en el conjunto de prueba")
print(classification_report(y_test, y_pred_test_rf))


# *********** Aquí termina la sección de agregar código *************

Bosque Aleatorio - Desempeño en el conjunto de entrenamiento
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1068
           1       0.99      1.00      1.00      1032

    accuracy                           1.00      2100
   macro avg       1.00      1.00      1.00      2100
weighted avg       1.00      1.00      1.00      2100

Bosque Aleatorio - Desempeño en el conjunto de validación
              precision    recall  f1-score   support

           0       0.70      0.78      0.74       216
           1       0.77      0.69      0.73       234

    accuracy                           0.73       450
   macro avg       0.74      0.74      0.73       450
weighted avg       0.74      0.73      0.73       450

Bosque Aleatorio - Desempeño en el conjunto de prueba
              precision    recall  f1-score   support

           0       0.69      0.77      0.73       216
           1       0.76      0.68      0.72       234

    accur

#**Pregunta - 9:**



Reporte del mejor modelo.


In [20]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


previous_results = {
    'Modelo': ['LR S4', 'RF S4', 'NB S4'],
    'Precisión': [0.85, 0.82, 0.82],
    'Recall': [0.84, 0.81, 0.81],
    'F1-Score': [0.84, 0.81, 0.81]
}

# Resultados actuales para regresión logística
logreg_results = {
    'Modelo': ['Regresión Logística'],
    'Precisión': [classification_report(y_test, y_pred_test, output_dict=True)['weighted avg']['precision']],
    'Recall': [classification_report(y_test, y_pred_test, output_dict=True)['weighted avg']['recall']],
    'F1-Score': [classification_report(y_test, y_pred_test, output_dict=True)['weighted avg']['f1-score']]
}

# Resultados actuales para bosque aleatorio
rf_results = {
    'Modelo': ['Bosque Aleatorio'],
    'Precisión': [classification_report(y_test, y_pred_test_rf, output_dict=True)['weighted avg']['precision']],
    'Recall': [classification_report(y_test, y_pred_test_rf, output_dict=True)['weighted avg']['recall']],
    'F1-Score': [classification_report(y_test, y_pred_test_rf, output_dict=True)['weighted avg']['f1-score']]
}

# Convertir resultados a DataFrames
previous_df = pd.DataFrame(previous_results)
logreg_df = pd.DataFrame(logreg_results)
rf_df = pd.DataFrame(rf_results)

# Combinar todos los resultados
comparison_df = pd.concat([previous_df, logreg_df, rf_df], axis=0, ignore_index=True)

# Mostrar la comparación
print("Comparación de Resultados")
print(comparison_df)


# *********** Aquí termina la sección de agregar código *************

Comparación de Resultados
                Modelo  Precisión    Recall  F1-Score
0                LR S4   0.850000  0.840000  0.840000
1                RF S4   0.820000  0.810000  0.810000
2                NB S4   0.820000  0.810000  0.810000
3  Regresión Logística   0.813755  0.813333  0.813400
4     Bosque Aleatorio   0.728740  0.724444  0.724161


#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**